In [1]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/WSTM-Proj

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1oWYt7oVJSbEFyIJzfizy8NTc6cl6XI09/WSTM-Proj


In [ ]:
from load import *
dname = 'Foursquare_small_small'
process_traj(dname)

100 81
200 193
300 11
400 95
500 35
600 12
700 11
800 177
900 9
1000 135
1100 45
1200 203
1300 6
1400 5
1500 4
1600 241
1700 82
1800 6
1900 93
2000 5
2100 49
2200 98
2300 56
2400 53
2500 42
2600 244
2700 57
2800 113
2900 52
3000 96
3100 37
3200 12
3300 17
3400 6
3500 57
3600 119
3700 145
3800 22
3900 34
4000 70
4100 8
4200 36
4300 113
4400 331
4500 57
4600 68
4700 139
4800 14
4900 106
5000 84
5100 624
5200 6
5300 50
5400 56
5500 14
5600 48
5700 25
5800 87
5900 36
6000 19
6100 50
6200 270
6300 54
6400 27
6500 733
6600 107
6700 80
6800 65
6900 164
7000 56
7100 5
7200 238
7300 137
7400 40
7500 37
7600 42
7700 14
7800 298
7900 18
8000 78
8100 90
8200 40
8300 29
8400 50
8500 85
8600 175
8700 5
8800 27
8900 69
9000 67
9100 177
9200 38
9300 349
9400 61
9500 54
9600 60
9700 69
9800 156
9900 75
10000 7
10100 14
10200 373
10300 15
10400 9
10500 84
10600 113
10700 32
10800 74
10900 30
11000 1320
11100 31
11200 74
11300 19
11400 11
11500 78
11600 52
11700 74
11800 4
11900 408
12000 121
12100 139
1

In [ ]:
# # old without sort 50-59s
# from load import *
# dname = 'Foursquare_small'
# process_traj(dname)

In [ ]:
# ~27 mins 33-34 epochs
from train import *

# load data
# dname = 'Foursquare_small'
print('./data/' + dname + '_data.pkl')
file = open('./data/' + dname + '_data.pkl', 'rb')
file_data = joblib.load(file)
# tensor(NUM, M, 3), np(NUM, M, M, 2), np(L, L), np(NUM, M, M), tensor(NUM, M), np(NUM)
[trajs, mat1, mat2s, mat2t, labels, lens, u_max, l_max] = file_data
mat1, mat2s, mat2t, lens = torch.FloatTensor(mat1), torch.FloatTensor(mat2s).to(device), \
                            torch.FloatTensor(mat2t), torch.LongTensor(lens)

# the run speed is very flow due to the use of location matrix (also huge memory cost)
# please use a partition of the data (recommended)
part = 100
trajs, mat1, mat2t, labels, lens = \
    trajs[:part], mat1[:part], mat2t[:part], labels[:part], lens[:part]

ex = mat1[:, :, :, 0].max(), mat1[:, :, :, 0].min(), mat1[:, :, :, 1].max(), mat1[:, :, :, 1].min()

stan = Model(t_dim=hours+1, l_dim=l_max+1, u_dim=u_max+1, embed_dim=256, ex=ex, dropout=0)
num_params = 0

for name in stan.state_dict():
    print(name)

for param in stan.parameters():
    num_params += param.numel()
print('num of params', num_params)

load = False

if load:
    checkpoint = torch.load('best_stan_win_' + dname + '.pth')
    stan.load_state_dict(checkpoint['state_dict'])
    start = time.time() - checkpoint['time']
    records = checkpoint['records']
else:
    records = {'epoch': [], 'acc_valid': [], 'acc_test': []}
    start = time.time()

trainer = Trainer(stan, records, load, trajs, mat1, mat2t, labels, lens, mat2s)
trainer.train(part, start, dname)
# trainer.inference()

./data/NYC_data.pkl


RuntimeError: ignored